# Test du Package Gomory depuis PyPI

Ce notebook démontre l'utilisation du package `gomory` installé depuis PyPI pour résoudre un Programme Linéaire en Nombres Entiers (PLNE) par la méthode des coupes de Gomory.

## Problème à résoudre

$$\max Z = 5x_1 + 8x_2$$

Sous contraintes :
$$\begin{cases}
x_1 + x_2 \leq 6 \\
5x_1 + 9x_2 \leq 45 \\
x_1, x_2 \geq 0 \text{ et entiers}
\end{cases}$$

## 1. Import du package gomory

In [ ]:
# Installation du package (décommenter si nécessaire)
# !pip install gomory


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import du package gomory installé depuis PyPI
from gomory import Problem, GomorySolver, display_tableau, display_cut

print("✅ Package gomory importé avec succès!")

✅ Package gomory importé avec succès!


## 2. Définition du problème PLNE

In [3]:
# Définition du problème PLNE
# Max Z = 5*x1 + 8*x2
# S.C:
#   x1 + x2 <= 6
#   5*x1 + 9*x2 <= 45
#   x1, x2 >= 0 et entiers

problem = Problem(
    objective=[5, 8],           # Coefficients de la fonction objectif
    sense="max",                # Maximisation
    constraints=[
        ([1, 1], "<=", 6),      # x1 + x2 <= 6
        ([5, 9], "<=", 45),     # 5*x1 + 9*x2 <= 45
    ],
    integer_vars=[0, 1],        # x1 et x2 doivent être entières
    var_names=["x1", "x2"]      # Noms des variables
)

print("📋 Problème défini:")
print(problem)

📋 Problème défini:
max z = 5*x1 + 8*x2
sous contraintes:
  x1 + x2 <= 6
  5*x1 + 9*x2 <= 45
  x1, x2 ∈ ℤ⁺


## 3. Résolution avec l'algorithme de Gomory

In [4]:
# Créer le solveur avec affichage détaillé des tableaux
def display_callback(iteration):
    print(f"\n{'='*70}")
    print(f"Itération {iteration.iteration_number}: {iteration.phase}")
    print(f"{'='*70}")
    print(display_tableau(iteration.tableau, title=iteration.description))
    
    if iteration.cut:
        print(display_cut(iteration.cut, iteration.tableau.var_names))

# Résoudre le problème
solver = GomorySolver(
    problem,
    verbose=True,
    display_callback=display_callback
)

result = solver.solve()

RÉSOLUTION PAR LA MÉTHODE DES COUPES DE GOMORY

max z = 5*x1 + 8*x2
sous contraintes:
  x1 + x2 <= 6
  5*x1 + 9*x2 <= 45
  x1, x2 ∈ ℤ⁺

Étape 1: Création du tableau initial (forme standard)

Itération 1: initialisation

                           Tableau initial                            
+----------+----------+----------+----------+----------+----------+----------+
|   Max    |    Ci    |    5     |    8     |    0     |    0     |          |
|    CB    |    B     |    b     |    x1    |    x2    |    x3    |    x4    |
+----------+----------+----------+----------+----------+----------+----------+
|    0     |    x3    |    6     |    1     |    1     |    1     |    0     |
|    0     |    x4    |    45    |    5     |    9     |    0     |    1     |
+----------+----------+----------+----------+----------+----------+----------+
|          |    Zi    |    0     |    0     |    0     |    0     |    0     |
|          |          |  Ci-Zi   |    5     |    8     |    0     |    0     

## 4. Résultat final et vérification

In [5]:
# Afficher le résultat final
print("=" * 70)
print("🎯 RÉSULTAT FINAL")
print("=" * 70)

if result.solution:
    x1, x2 = result.solution[0], result.solution[1]
    z = result.optimal_value
    
    print(f"\n✅ Solution optimale entière trouvée!")
    print(f"\n   x1* = {x1}")
    print(f"   x2* = {x2}")
    print(f"   Z*  = {z}")
    
    print(f"\n📊 Statistiques:")
    print(f"   Nombre de coupes de Gomory: {result.num_cuts}")
    print(f"   Nombre total d'itérations: {result.total_iterations}")
    
    # Vérification des contraintes
    print(f"\n🔍 Vérification des contraintes:")
    c1 = x1 + x2
    c2 = 5*x1 + 9*x2
    print(f"   x1 + x2 = {c1} <= 6  ✓" if c1 <= 6 else f"   x1 + x2 = {c1} > 6  ✗")
    print(f"   5*x1 + 9*x2 = {c2} <= 45  ✓" if c2 <= 45 else f"   5*x1 + 9*x2 = {c2} > 45  ✗")
    
    # Vérification d'intégrité
    print(f"\n🔢 Vérification d'intégrité:")
    print(f"   x1 = {x1} est entier  ✓" if x1 == int(x1) else f"   x1 = {x1} n'est pas entier  ✗")
    print(f"   x2 = {x2} est entier  ✓" if x2 == int(x2) else f"   x2 = {x2} n'est pas entier  ✗")
else:
    print(f"Statut: {result.status}")

🎯 RÉSULTAT FINAL

✅ Solution optimale entière trouvée!

   x1* = 0
   x2* = 5
   Z*  = 40

📊 Statistiques:
   Nombre de coupes de Gomory: 1
   Nombre total d'itérations: 5

🔍 Vérification des contraintes:
   x1 + x2 = 5 <= 6  ✓
   5*x1 + 9*x2 = 45 <= 45  ✓

🔢 Vérification d'intégrité:
   x1 = 0 est entier  ✓
   x2 = 5 est entier  ✓
